In [19]:
import src.data_mgmt.RecSysData as rsd
from torch.utils.data import DataLoader
import torch
import pandas as pd
from src.data_mgmt.BaseDataClass import BaseDataClass as bdc

In [21]:
#def _loadUpDf(pathname, filename, extensions):
           
df = bdc._loadUpDf('/Users/andrealevy/cmu_msba_2022_ml_applications_2/data/','train','.json.gz')

Loading JSON into Dataframe: 200000it [00:14, 13886.61it/s]


In [23]:
df.columns

Index(['reviewHash', 'reviewTime', 'reviewText', 'nHelpful', 'outOf',
       'reviewerID', 'reviewHash', 'categories', 'unixReviewTime', 'itemID',
       'rating', 'summary', 'categoryID', 'helpfulPerc',
       'summaryCharacterLength', 'reviewTextCharacterLength', 'parentCategory',
       'cat1', 'cat2', 'cat3', 'cat1_child', 'cat2_parent', 'cat2_child'],
      dtype='object')

In [25]:
df.head()

,reviewHash,reviewTime,reviewText,nHelpful,outOf,reviewerID,reviewHash,categories,unixReviewTime,itemID,...,helpfulPerc,summaryCharacterLength,reviewTextCharacterLength,parentCategory,cat1,cat2,cat3,cat1_child,cat2_parent,cat2_child
0,R798569390,2013-09-26,The model in this picture has them rolled up a...,0,0,U490934656,R798569390,"[[Clothing, Shoes & Jewelry, Women], [Clothing...",1380153600,I402344648,...,NaN,12,276,"Clothing, Shoes & Jewelry",Women,,,Women,"Clothing, Shoes & Jewelry",Leggings
1,R436443063,2013-02-07,"I love the look of this bra, it is what I want...",0,0,U714157797,R436443063,"[[Clothing, Shoes & Jewelry, Women, Clothing, ...",1360195200,I697650540,...,NaN,29,269,"Clothing, Shoes & Jewelry",Women,Clothing,"Lingerie, Sleep & Lounge",Everyday Bras,"Clothing, Shoes & Jewelry",Everyday Bras
2,R103439446,2014-03-16,I am not comfortable with wearing my wedding b...,0,0,U507366950,R103439446,"[[Clothing, Shoes & Jewelry, Wedding Party Gif...",1394928000,I464613034,...,NaN,28,1073,"Clothing, Shoes & Jewelry",Wedding Party Gifts,Gifts for the Groomsmen,,Gifts for the Groomsmen,"Clothing, Shoes & Jewelry",Women
3,R486351639,2014-03-10,Like the look of this top and really looks cut...,0,0,U307862152,R486351639,"[[Clothing, Shoes & Jewelry, Women, Clothing, ...",1394409600,I559560885,...,NaN,32,464,"Clothing, Shoes & Jewelry",Women,Clothing,Swimsuits & Cover Ups,Cover-Ups,,
4,R508664275,2013-07-30,I'm quite small and the XS fits me like a regu...,1,1,U742726598,R508664275,"[[Clothing, Shoes & Jewelry, Women, Plus-Size,...",1375142400,I476005312,...,1.0,11,424,"Clothing, Shoes & Jewelry",Women,Plus-Size,Tops & Tees,Knits & Tees,"Clothing, Shoes & Jewelry",T-Shirts


In [26]:
ppath='/Users/andrealevy/cmu_msba_2022_ml_applications_2/data/'

tt = rsd.RecSysData(ppath)

Loading JSON into Dataframe: 200000it [00:14, 14071.45it/s]


In [30]:
tt.df_data.head()

,reviewHash,reviewHash,reviewerID,unixReviewTime,itemID,rating,uid,pid
0,R798569390,R798569390,U490934656,1380153600,I402344648,4.0,0,0
1,R436443063,R436443063,U714157797,1360195200,I697650540,4.0,1,1
2,R103439446,R103439446,U507366950,1394928000,I464613034,5.0,2,2
3,R486351639,R486351639,U307862152,1394409600,I559560885,2.0,3,3
4,R508664275,R508664275,U742726598,1375142400,I476005312,5.0,4,4


In [31]:
print(f"There are {tt.df_data.uid.unique().shape} unique users and...")
print(f"There are {tt.df_data.pid.unique().shape} unique products out of ...")
print(f"A total of {tt.df_data.shape} records in the training dataset.")

There are (39239,) unique users and...
There are (19914,) unique products out of ...
A total of (200000, 8) records in the training dataset.


In [32]:
b_size = 4

train_loader = DataLoader(tt, batch_size=b_size, shuffle=True)

Questions: All users in the dataset purchased the item already, how do we want to determine in test set if item will be purchased or not

Baseline looks at number of times and item is purchased and if item is in the top 50% of pruchases, predict that test user will purchase.


Ideas: Top highly rated items? 

User Rec Sys - > if we predict higher than 3.5, user will purchase?